In [1]:
import os
## GAN Variants
# from GAN import GAN
# from CGAN import CGAN
# from infoGAN import infoGAN
# from ACGAN import ACGAN
# from EBGAN import EBGAN
# from WGAN import WGAN
from WGAN_GP_withWM import WGAN_GP
# from DRAGAN import DRAGAN
# from LSGAN import LSGAN
# from BEGAN import BEGAN

## VAE Variants
# from VAE import VAE
# from CVAE import CVAE

from utils_withWM import show_all_variables
from utils_withWM import check_folder

import tensorflow as tf
import argparse

import numpy as np

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9  # 程序最多只能占用指定gpu90%的显存
config.gpu_options.allow_growth = True      #程序按需申请内存
sess = tf.Session(config = config)

In [2]:
"""parsing and configuration"""
def parse_args():
    desc = "Tensorflow implementation of GAN collections"
    parser = argparse.ArgumentParser(description=desc)
    
    #args = parser.parse_args(args=[])
    
    parser.add_argument('--gan_type', type=str, default='GAN',
                        choices=['GAN', 'CGAN', 'infoGAN', 'ACGAN', 'EBGAN', 'BEGAN', 'WGAN', 'WGAN_GP', 'DRAGAN', 'LSGAN', 'VAE', 'CVAE'],
                        help='The type of GAN')
                        #help='The type of GAN', required=True)
    parser.add_argument('--dataset', type=str, default='mnist', choices=['mnist', 'fashion-mnist', 'celebA', 'Anime'],
                        help='The name of dataset')
    parser.add_argument('--epoch', type=int, default=100, help='The number of epochs to run')
    parser.add_argument('--batch_size', type=int, default=128, help='The size of batch')
    parser.add_argument('--z_dim', type=int, default=128, help='Dimension of noise vector')
    parser.add_argument('--checkpoint_dir', type=str, default='./ResultWithWM/checkpoint',
                        help='Directory name to save the checkpoints')
    parser.add_argument('--result_dir', type=str, default='./ResultWithWM/results',
                        help='Directory name to save the generated images')
    parser.add_argument('--log_dir', type=str, default='./ResultWithWM/logs',
                        help='Directory name to save training logs')
    parser.add_argument('--mode', type=str, default='train',
                        help='train or infer')
    parser.add_argument('--testing_tags', type=str, default='./AnimeDataset/sample_testing_text.txt',
                        help='train or infer')## 测试数据存放处
    return check_args(parser.parse_args(args=['--gan_type','WGAN_GP','--dataset','Anime','--mode','infer',
                                              '--testing_tags','./extra_data/test_wm.csv']))
    #return check_args(parser.parse_args())

"""checking arguments"""
def check_args(args):
    # --checkpoint_dir
    check_folder(args.checkpoint_dir)

    # --result_dir
    check_folder(args.result_dir)

    # --result_dir
    check_folder(args.log_dir)

    # --epoch
    assert args.epoch >= 1, 'number of epochs must be larger than or equal to one'

    # --batch_size
    assert args.batch_size >= 1, 'batch size must be larger than or equal to one'

    # --z_dim
    assert args.z_dim >= 1, 'dimension of noise vector must be larger than or equal to one'

    return args

In [3]:
"""main"""
def main():
    # Fix random seed.
    tf.set_random_seed(9487)
    #tf.random.set_seed(9487)
    
    # parse arguments
    args = parse_args()
    if args is None:
          exit()

    # open session
    # models = [GAN, CGAN, infoGAN, ACGAN, EBGAN, WGAN, WGAN_GP, DRAGAN,
    #           LSGAN, BEGAN, VAE, CVAE]
    models = [WGAN_GP] 
    #with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:    
        # declare instance for GAN

        gan = None
        for model in models:
            if args.gan_type == model.model_name:
                gan = model(sess,
                            epoch=args.epoch,
                            batch_size=args.batch_size,
                            z_dim=args.z_dim,
                            dataset_name=args.dataset,
                            checkpoint_dir=args.checkpoint_dir,
                            result_dir=args.result_dir,
                            log_dir=args.log_dir, 
                            mode=args.mode)
        if gan is None:
            raise Exception("[!] There is no option for " + args.gan_type)

        # build graph
        gan.build_model()

        # show network architecture
        show_all_variables()
        if args.mode == 'train':
            # launch the graph in a session
            gan.train()
            print(" [*] Training finished!")

            # # visualize learned generator
            # gan.visualize_results(args.epoch-1)
            # print(" [*] Testing finished!")
        elif args.mode == 'infer':
            tag_dict = ['orange hair', 'white hair', 'aqua hair', 'gray hair', 'green hair', 'red hair', 'purple hair', 
                        'pink hair', 'blue hair', 'black hair', 'brown hair', 'blonde hair', 'no hair',
                        'gray eyes', 'black eyes', 'orange eyes', 'pink eyes', 'yellow eyes',
                        'aqua eyes', 'purple eyes', 'green eyes', 'brown eyes', 'red eyes', 'blue eyes','no eyes']
            testing_tags_txt = open(args.testing_tags, 'r').readlines()

            test_labels = np.zeros((args.batch_size, len(tag_dict)))
            for line in testing_tags_txt:
                id, tags = line.split(',')
                #id, tags = line.split('.')
                #tags = tags.replace('\n','').split('/')
                #print(tags)
                label = np.zeros(len(tag_dict))
                #count=0
                for i in range(len(tag_dict)):
                    if tag_dict[i] in tags:
                        label[i] = 1
                       # count = count + 1
                test_labels[int(id) - 1] = label
                #print(count)
            # visualize learned generator
            gan.infer(test_labels)
            print(" [*] Infer finished!")

print("finish")

finish


In [4]:
if __name__ == '__main__':
    main()


W0326 01:59:16.331705 140666994300736 deprecation_wrapper.py:119] From /tf/notebooks/anime_myy/WGAN_GP_withWM.py:160: The name tf.set_random_seed is deprecated. Please use tf.compat.v1.set_random_seed instead.

W0326 01:59:16.333045 140666994300736 deprecation_wrapper.py:119] From /tf/notebooks/anime_myy/WGAN_GP_withWM.py:168: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0326 01:59:16.387472 140666994300736 deprecation_wrapper.py:119] From /tf/notebooks/anime_myy/WGAN_GP_withWM.py:173: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



[Tip 14] Train discriminator more (sometimes)
[Tip Hung-yi Lee] Data Augmentation (flip images horizontally + rotate images)


W0326 01:59:16.435742 140666994300736 deprecation_wrapper.py:119] From /tf/notebooks/anime_myy/WGAN_GP_withWM.py:132: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.



[Tip 4] BatchNorm
[Tip 5] Avoid Sparse Gradients: ReLU, MaxPool


W0326 01:59:16.633753 140666994300736 deprecation.py:323] From /tf/notebooks/anime_myy/WGAN_GP_withWM.py:141: batch_normalization (from tensorflow.python.layers.normalization) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
W0326 01:59:17.297749 140666994300736 deprecation_wrapper.py:119] From /tf/notebooks/anime_myy/WGAN_GP_withWM.py:197: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.



...

[Tip 5] Avoid Sparse Gradients: ReLU, MaxPool
---------
Variables: name (type shape) [size]
---------
generator/fully_connected/weights:0 (float32_ref 153x6144) [940032, bytes: 3760128]
generator/fully_connected/biases:0 (float32_ref 6144) [6144, bytes: 24576]
generator/batch_normalization/gamma:0 (float32_ref 6144) [6144, bytes: 24576]
generator/batch_normalization/beta:0 (float32_ref 6144) [6144, bytes: 24576]
generator/Conv2d_transpose/weights:0 (float32_ref 5x5x256x384) [2457600, bytes: 9830400]
generator/Conv2d_transpose/biases:0 (float32_ref 256) [256, bytes: 1024]
generator/batch_normalization_1/gamma:0 (float32_ref 256) [256, bytes: 1024]
generator/batch_normalization_1/beta:0 (float32_ref 256) [256, bytes: 1024]
generator/Conv2d_transpose_1/weights:0 (float32_ref 5x5x128x256) [819200, bytes: 3276800]
generator/Conv2d_transpose_1/biases:0 (float32_ref 128) [128, bytes: 512]
generator/batch_normalization_2/gamma:0 (float32_ref 128) [128, bytes: 512]
generator/batch_normalization

W0326 01:59:20.120452 140666994300736 deprecation_wrapper.py:119] From /tf/notebooks/anime_myy/WGAN_GP_withWM.py:430: The name tf.train.Saver is deprecated. Please use tf.compat.v1.train.Saver instead.

W0326 01:59:20.194525 140666994300736 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


 [*] Reading checkpoints...
 [*] Success to read WGAN_GP.model-28801


W0326 01:59:21.424207 140666994300736 util.py:42] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


 [*] Load SUCCESS
 [*] Infer finished!


In [5]:
import gc
gc.collect()

0